⊕ [Basic Usage of python-rocksdb — python-rocksdb 0.6.7 documentation](https://python-rocksdb.readthedocs.io/en/latest/tutorial/index.html#iteration)

* A more production ready open can look like this
* 可以通过配置 block-cache 来提升读取性能，特别是在有热点的情况下，效果非常好。RocksDB 的 block-cache 分为 uncompressed block-cache 和 compressed block-cache，前者用于存储解压后的 block（其他数据库里面叫 page），后者存储压缩状态下的 block。当只配置 uncompressed block-cache 的情况下，一般采用系统的 page cache 来充当 compressed block-cache。如果配置了 compressed block-cache，可以通过设置 RocksDB 的 use_direct_reads 参数来使用 direct io 读，这样就绕过系统的 page cache，防止 page cache 和 compressed block-cache 作用重复了。另外，RocksDB 和 LevelDB 一样，每个 sst 都会有一个 bloom filter，用于加速查询。

```python
import rocksdb

opts = rocksdb.Options()
opts.create_if_missing = True
opts.max_open_files = 300000
opts.write_buffer_size = 67108864
opts.max_write_buffer_number = 3
opts.target_file_size_base = 67108864

opts.table_factory = rocksdb.BlockBasedTableFactory(
    filter_policy=rocksdb.BloomFilterPolicy(10),
    block_cache=rocksdb.LRUCache(2 * (1024 ** 3)),
    block_cache_compressed=rocksdb.LRUCache(500 * (1024 ** 2)))

db = rocksdb.DB("test.db", opts)
```

In [2]:
import rocksdb

class StaticPrefix(rocksdb.interfaces.SliceTransform):
    def name(self):
        return b'static'

    def transform(self, src):
        return (0, 5)

    def in_domain(self, src):
        return len(src) >= 5

    def in_range(self, dst):
        return len(dst) == 5


opts = rocksdb.Options()
opts.prefix_extractor = StaticPrefix()
opts.allow_concurrent_memtable_write = False
opts.memtable_factory = rocksdb.HashSkipListMemtableFactory()
opts.create_if_missing = True

db = rocksdb.DB("test.db", opts)
db.put(b'00001.x', b'x')
db.put(b'00001.y', b'y')
db.put(b'00002.x', b'x')

In [3]:
db.put(b'00002.x', b'x')
db.put(b'00002.y', b'y')
db.put(b'00002.z', b'z')

db.put(b'00003.x', b'x')
db.put(b'00003.y', b'y')
db.put(b'00003.z', b'z')

In [7]:
import itertools

prefix = b'00002'

it = db.iteritems()
it.seek(prefix)
dict(itertools.takewhile(lambda item: item[0].startswith(prefix), it))

{b'00002.x': b'x', b'00002.y': b'y', b'00002.z': b'z'}